# Convert a PyTorch model to Tensorflow using ONNX

In this tutorial, we will show you how to export a model defined in PyTorch to ONNX and then import the ONNX model into Tensorflow to run it. We will also show you how to save this Tensorflow model into a file for later use.  

## Installations

First let's install [ONNX](https://github.com/onnx/onnx), [PyTorch](https://github.com/pytorch/pytorch), and [Tensorflow](https://github.com/tensorflow/tensorflow) by following the instructions on each of their repository.

Next install [onnx-tensorflow](https://github.com/onnx/onnx-tensorflow) by the following commands:
```
git clone git@github.com:onnx/onnx-tensorflow.git && cd onnx-tensorflow
pip install -e .
```

## Define model

In this tutorial we are going to use the [MNIST model](https://github.com/pytorch/examples/tree/master/mnist) from PyTorch examples.  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Train and test model

In [2]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

# Train this model with 60 epochs and after process every 300 batches log the train status 
args = parser.parse_args(['--epochs', '60', '--log-interval', '300'])

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.342948
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.945489
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.590510
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.371047

Test set: Average loss: 0.2032, Accuracy: 9407/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.371472
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.316857
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.205765
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.423041

Test set: Average loss: 0.1220, Accuracy: 9617/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.489221
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.504540
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.205699
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.385294

Test set: Average loss: 0.1024, Accuracy: 9676/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.399744
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.288648
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.144865
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.082947

Test set: Averag

Train Epoch: 31 [57600/60000 (96%)]	Loss: 0.199190

Test set: Average loss: 0.0358, Accuracy: 9894/10000 (99%)

Train Epoch: 32 [0/60000 (0%)]	Loss: 0.112976
Train Epoch: 32 [19200/60000 (32%)]	Loss: 0.080692
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.531600
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.092054

Test set: Average loss: 0.0364, Accuracy: 9888/10000 (99%)

Train Epoch: 33 [0/60000 (0%)]	Loss: 0.112255
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.173839
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.100733
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.212895

Test set: Average loss: 0.0338, Accuracy: 9888/10000 (99%)

Train Epoch: 34 [0/60000 (0%)]	Loss: 0.181373
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.091187
Train Epoch: 34 [38400/60000 (64%)]	Loss: 0.146965
Train Epoch: 34 [57600/60000 (96%)]	Loss: 0.030407

Test set: Average loss: 0.0343, Accuracy: 9891/10000 (99%)

Train Epoch: 35 [0/60000 (0%)]	Loss: 0.203500
Train Epoch: 35 [19200/60000 (32%)]	Loss: 0.347259
Train Epoch

## Save the trained model

In [3]:
# Save the trained model to a file
torch.save(model.state_dict(), 'mnist.pth')

## Export the trained model to ONNX 
In order to export the model, it needs to run the model once and save this resulting traced model to a ONNX file. Therefore, we need to provide the input for the MNIST model

In [4]:
from torch.autograd import Variable

# Load the trained model from file
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")

## Import the ONNX model to Tensorflow

In [5]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('mnist.onnx')

# Verify the ONNX model is well formed
onnx.checker.check_model(model)

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

Fail to get since_version of Expand in domain `` with max_inclusive_version=7. Set to 1.


Let's explore the tf_rep object return from onnx.tf.backend.prepare

In [6]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

inputs: ['0']
outputs: ['22']
tensor_dict:
{'12': <tf.Tensor 'Add_1:0' shape=(1, 20, 8, 8) dtype=float32>, '17': <tf.Tensor 'add_3:0' shape=(1, 50) dtype=float32>, '9': <tf.Tensor 'Add:0' shape=(1, 10, 24, 24) dtype=float32>, '3': <tf.Tensor 'Const_2:0' shape=(20, 10, 5, 5) dtype=float32>, '5': <tf.Tensor 'Const_4:0' shape=(50, 320) dtype=float32>, '4': <tf.Tensor 'Const_3:0' shape=(20,) dtype=float32>, '21': <tf.Tensor 'add_4:0' shape=(1, 10) dtype=float32>, '1': <tf.Tensor 'Const:0' shape=(10, 1, 5, 5) dtype=float32>, '18': <tf.Tensor 'Relu_2:0' shape=(1, 50) dtype=float32>, '2': <tf.Tensor 'Const_1:0' shape=(10,) dtype=float32>, '14': <tf.Tensor 'Relu_1:0' shape=(1, 20, 4, 4) dtype=float32>, '15': <tf.Tensor 'Const_14:0' shape=(2,) dtype=int64>, '13': <tf.Tensor 'max_pool_1:0' shape=(1, 20, 4, 4) dtype=float32>, '16': <tf.Tensor 'Reshape:0' shape=(1, 320) dtype=float32>, '11': <tf.Tensor 'Relu:0' shape=(1, 10, 12, 12) dtype=float32>, '6': <tf.Tensor 'Const_5:0' shape=(50,) dtype=flo

## Run the model in Tensorflow

In [7]:
import numpy as np
from IPython.display import display
from PIL import Image

print('Image 1:')
img = Image.open('assets/two.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('output = ',  output._0)
print('The digit is classified as ', np.argmax(output))

print('Image 2:')
img = Image.open('assets/three.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('output = ',  output._0)
print('The digit is classified as ', np.argmax(output))

Image 1:


output =  [[-142.03171  -172.64523     0.        -61.611305 -178.14746  -176.38095
  -149.58891  -169.29485   -96.73183  -165.3873  ]]
The digit is classified as  2
Image 2:


output =  [[-105.85771 -124.99865    0.       -21.94822 -152.93634  -68.16041
   -86.98468 -134.10194  -72.43932 -145.65083]]
The digit is classified as  2


## Save the model into a file

In [8]:
tf_rep.export_graph('mnist.pb')